In [5]:
import pathlib

from compas.files import OBJ
from compas.datastructures import Mesh
from compas_model.model import Model
from compas_model.elements import BlockElement

filepath = pathlib.Path().home() / "Code" / "compas_model" / "data" / "wall_stack.obj"

obj = OBJ(filepath)
obj.read()

meshes = []
for name in obj.objects:  # type: ignore
    vertices, faces = obj.objects[name]  # type: ignore
    mesh = Mesh.from_vertices_and_faces(vertices, faces)
    mesh.name = name
    meshes.append(mesh)

model = Model()

for mesh in meshes:
    block = BlockElement(shape=mesh)
    model.add_element(block)


In [10]:
from compas_model.algorithms import collider

interfaces = []
for item in collider.get_collision_pairs(model, tolerance_area=0.1):
    i, j, collisions = item
    for faces, polygon in collisions:
        interfaces.append(polygon)
        model.add_interaction(model.elementlist[i], model.elementlist[j])

In [11]:
model.print()

Model Hierarchy
└── <TreeNode root>
    ├── <TreeNode ElementNode>
    ├── <TreeNode ElementNode>
    ├── <TreeNode ElementNode>
    ├── <TreeNode ElementNode>
    ├── <TreeNode ElementNode>
    └── <TreeNode ElementNode>
Model Interactions
0
- 1: None
1
- 0: None
- 2: None
2
- 1: None
- 3: None
3
- 2: None
- 4: None
4
- 3: None
- 5: None
5
- 4: None


In [13]:
from compas_notebook.viewer import Viewer
from compas.colors import Color

viewer = Viewer()

for element in model.elementlist:
    viewer.scene.add(element, color=Color.from_hex('#cccccc'), show_faces=False, show_edges=True)

for polygon in interfaces:
    viewer.scene.add(polygon, color=Color.red())

viewer.show()